In [2]:
import pandas as pd
import numpy as np
import os
import glob
from collections import Counter
import math
from bitarray import bitarray
import mmh3
import matplotlib.pyplot as plt
from __future__ import division
plt.style.use('ggplot')
%matplotlib inline

In [1]:
import numpy as np 
import pandas as pd 
# read data and assign column names 
cols = (
   'date', 'flow_start', 'duration', 'protocol', 'src_addrIP',
    'direction', 'dst_addrIP', 'flags', 'tos', 'packets',
    'bytes', 'flows', 'label', 
)
dataset = pd.read_csv('capture20110818.pcap.netflow.labeled', header=0,names=cols,skiprows=1, sep='\s+')
df_ip = dataset.iloc[:,6]
iplist = df_ip.tolist()


In [3]:
#create two lists list1 and list2
list1 = int(math.floor((len(iplist))/2))
list2 = int(math.ceil((len(iplist))/2))
training = iplist[:list1]
test = iplist[list2:]

print(len(training) + len(test))
print(len(training))

5180850
2590425


In [4]:
#initialize l as a list of sequence 
l = [1,2,3,2,4,3,2]


In [6]:
#create an empty set to map the test and train ips and store them
match_test_train_ip = {}
intr = list(set(training).intersection(set(test)))
print('Number of IP addresses that are same for Test and Training dataset: ', len(intr))

Number of IP addresses that are same for Test and Training dataset:  124800


In [11]:
!pip install -U mmh3

  Using cached https://files.pythonhosted.org/packages/fa/7e/3ddcab0a9fcea034212c02eb411433db9330e34d626360b97333368b4052/mmh3-2.5.1.tar.gz
  Running setup.py bdist_wheel for mmh3: started
  Running setup.py bdist_wheel for mmh3: finished with status 'done'
  Stored in directory: C:\Users\chitra\AppData\Local\pip\Cache\wheels\38\b4\ea\6e4e321c625d3320c0c496bf4088371546d8fce5f1dd71b219
Successfully built mmh3


In [7]:
#function to generate a bloom filter say we create 3 sample sizes mentioned in the tutorial 
def bloom_filter ( n ):
    bit_array = bitarray(n)
    bit_array.setall(0)
    for i in training:
        b1 = mmh3.hash(str(i), 42) % n 
        bit_array[b1] = 1
        b2 = mmh3.hash(str(i), 24) % n 
        bit_array[b2] = 1
        b3 = mmh3.hash(str(i), 98) % n 
        bit_array[b3] = 1  
    return bit_array

In [9]:
# a function that calculates fp! if the values in all bits =1 then we store it in a found list and subtract it from the total list 
def calculate_false_positives ( bit_array, n ):
    found_list = []
    false_positive_count = 0
    for i in test:
        b1 = mmh3.hash(i, 42) % n 
        b2 = mmh3.hash(i, 24) % n 
        b3 = mmh3.hash(i, 98) % n
        x = bit_array[b1] == 1 and bit_array[b2] == 1 and bit_array[b3] == 1
        if x:
            found_list.append(i)
    false_positive_list = list(set(found_list) - set(intr))
    return false_positive_list

In [10]:
#false positive rate can be computed by the below metioned formula 
def false_positive_rate_theoretical_bound ( m, n ):
    t0 = (m * np.log(2)) / n
    t1 = - t0 * (n / m)
    t2 = np.exp(t1)
    t3 = 1 - t2
    t4 = np.power(t3, t0)
    print("t4 = ",t4)

In [11]:
# now by taking some random value of n we try our pf rate 
n = 10000
bit_array_10k = bloom_filter(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_10k, n)))
print("false positive rate: ", false_positive_rate_theoretical_bound(n, 361423))

number of false positives:  231100
t4 =  0.9867945953273449
false positive rate:  None


In [12]:
# keep increasing n exponentaially as we dont know where it would stop 
n = 100000
bit_array_100k = bloom_filter(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_100k, n)))
print("false positive rate: ", false_positive_rate_theoretical_bound(n, 361423))

number of false positives:  231066
t4 =  0.8755231258941681
false positive rate:  None


In [13]:
#increse n again 

n = 1000000
bit_array_1M = bloom_filter(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_1M, n)))
print("false positive rate: ", false_positive_rate_theoretical_bound(n, 361423))

number of false positives:  57992
t4 =  0.2646526331801972
false positive rate:  None


In [14]:
# increase 

n = 10000000
bit_array_10M = bloom_filter(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_10M, n)))
print("false positive rate: ", false_positive_rate_theoretical_bound(n, 361423))

number of false positives:  190
t4 =  1.6856297745011987e-06
false positive rate:  None


In [15]:
# we can see that the numer of fp are drastically decresing we can stop here

n = 15000000
bit_array_15M = bloom_filter(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_15M, n)))
print("false positive rate: ", false_positive_rate_theoretical_bound(n, 361423))

number of false positives:  72
t4 =  2.188483571943762e-09
false positive rate:  None


In [16]:
l2 = np.log(2) * np.log(2)
# We choose p = 0.01
lp = np.log(0.01)

In [17]:
# Get number of distinct IP addresses in training data
n = len(set(training))
print(n)
# Size of bit array
m = - ( n * lp ) / l2
print("bit array size: ", m)

# No of hash functions to use 
k = - np.log(0.001) / np.log(2)
print("number of hash functions: ", k)

332038
bit array size:  3182603.6135043297
number of hash functions:  9.965784284662087


In [19]:
#now we cam create many bit arrays with varying sample size
def bloom_filter_10 ( n ):
    bit_array = bitarray(n)
    bit_array.setall(0)
    for i in training:
        b1 = mmh3.hash(str(i), 42) % n 
        bit_array[b1] = 1
        b2 = mmh3.hash(str(i), 27) % n 
        bit_array[b2] = 1
        b3 = mmh3.hash(str(i), 89) % n 
        bit_array[b3] = 1  
        b4 = mmh3.hash(str(i), 35) % n 
        bit_array[b4] = 1  
        b5 = mmh3.hash(str(i), 16) % n 
        bit_array[b5] = 1  
        b6 = mmh3.hash(str(i), 68) % n 
        bit_array[b6] = 1  
        b7 = mmh3.hash(str(i), 75) % n 
        bit_array[b7] = 1  
        b8 = mmh3.hash(str(i), 29) % n 
        bit_array[b8] = 1  
        b9 = mmh3.hash(str(i), 8) % n 
        bit_array[b9] = 1  
        b10 = mmh3.hash(str(i), 53) % n 
        bit_array[b10] = 1  
    return bit_array

In [20]:
#self explainatory 
def calculate_false_positives_10 ( bit_array, n ):
    found_list = []
    false_positive_count = 0
    for i in test:
        b1 = mmh3.hash(i, 42) % n 
        b2 = mmh3.hash(i, 24) % n 
        b3 = mmh3.hash(i, 98) % n
        x = bit_array[b1] == 1 and bit_array[b2] == 1 and bit_array[b3] == 1 and bit_array[b4] and bit_array[b5] and bit_array[b6] and bit_array[b7] and  bit_array[b8] and bit_array[b9] and bit_array[b10] 
        if x:
            found_list.append(i)
    false_positive_list = list(set(found_list) - set(intr))
    return false_positive_list

In [21]:
n = 3464261
bit_array_optimal = bloom_filter_10(n)
print("number of false positives: ", len(calculate_false_positives (bit_array_optimal, n)))

number of false positives:  53961


In [22]:
n = len(set(training)) + len(set(test))
print(n)

687938


In [33]:
# as mentioned in a source(https://en.wikipedia.org/wiki/Count%E2%80%93min_sketch) the method of calculation of w and d.
e = 2.718281828

#initialize ε and δ
for epsilon in [0.0001, 0.001, 0.01]:
    for delta in [0.0001, 0.001, 0.01]:
        # derive w and d
        w = round(e/epsilon)
        d = round(math.log(1/delta))
        print("w:",w)
        print("d:",d)

w: 27183
d: 9
w: 27183
d: 7
w: 27183
d: 5
w: 2718
d: 9
w: 2718
d: 7
w: 2718
d: 5
w: 272
d: 9
w: 272
d: 7
w: 272
d: 5


In [23]:
#as per question we can vary the values of w and d kindly take the values that are estimated in the above step for computation 
w = 10000
d = 9
mat = np.zeros([d, w])
seeds = np.random.randint(1, 100, d, dtype='l')

In [26]:
def CM_Sketch ( w, d ):  
    for ip in iplist:
        for i in range(d):
            b = mmh3.hash(str(ip),seeds[i]) % w
            mat[i][b] += 1

In [25]:
CM_Sketch(w, d)
print(mat.shape)

(9, 10000)


In [27]:
def get_count (ip, w, d):
    counts = np.zeros(d)
    for i in range(d):
        b = mmh3.hash(ip, seeds[i]) % w 
        counts[i] = mat[i][b]
        # print "b, freq", b, mat[i][b]
    return np.ndarray.min(counts)

In [28]:
ip_freq = {}
for ip in set(iplist):
    ip_freq[ip] = get_count(ip, w, d)

In [29]:
t = [('147.32.84.229:13363', 516869),
('147.32.80.9:53', 497651),
('147.32.85.123:13815', 48768),
('147.32.84.229:443', 44586),
('147.32.80.13:80', 30602),
('147.32.84.118:6881', 20378),
('147.32.84.2:80', 18109),
('76.13.114.90:80', 15863),
('74.125.232.213:443', 14544),
('147.32.86.58:48784', 12389)]

In [30]:
t

[('147.32.84.229:13363', 516869),
 ('147.32.80.9:53', 497651),
 ('147.32.85.123:13815', 48768),
 ('147.32.84.229:443', 44586),
 ('147.32.80.13:80', 30602),
 ('147.32.84.118:6881', 20378),
 ('147.32.84.2:80', 18109),
 ('76.13.114.90:80', 15863),
 ('74.125.232.213:443', 14544),
 ('147.32.86.58:48784', 12389)]

In [31]:
for row in range(10):
    print(t[row][0], t[row][1], ip_freq[t[row][0]])

147.32.84.229:13363 516869 697825.0
147.32.80.9:53 497651 482849.0
147.32.85.123:13815 48768 166.0
147.32.84.229:443 44586 64301.0
147.32.80.13:80 30602 59114.0
147.32.84.118:6881 20378 8082.0
147.32.84.2:80 18109 12995.0
76.13.114.90:80 15863 15224.0
74.125.232.213:443 14544 8289.0
147.32.86.58:48784 12389 4789.0
